task name: MapKeyToVector
task description: Implement the MapKeyToVector example using ML.NET

In [ ]:
#r "nuget:Microsoft.ML,1.6.0"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext();

In [ ]:
public class DataPoint
{
    [KeyType(9)]
    public uint Timeframe { get; set; }
    public int PartA { get; set; }
    public int PartB { get; set; }
}

var rawData = new[] {
    new DataPoint() { Timeframe = 8, PartA=1, PartB=2},
    new DataPoint() { Timeframe = 7, PartA=2, PartB=1},
    new DataPoint() { Timeframe = 8, PartA=3, PartB=2},
    new DataPoint() { Timeframe = 3, PartA=3, PartB=3}
};
var data = mlContext.Data.LoadFromEnumerable(rawData);

In [ ]:
var pipeline = mlContext.Transforms.Conversion.MapKeyToVector(
    "TimeframeVector", "Timeframe")
    .Append(mlContext.Transforms.Concatenate("Parts", "PartA", "PartB"))
    .Append(mlContext.Transforms.Conversion.MapValueToKey("Parts"))
    .Append(mlContext.Transforms.Conversion.MapKeyToVector(
        "PartsCount", "Parts", outputCountVector: true))
    .Append(mlContext.Transforms.Conversion.MapKeyToVector(
        "PartsNoCount", "Parts"));

In [ ]:
IDataView transformedData = pipeline.Fit(data).Transform(data);

In [ ]:
public class TransformedData : DataPoint
{
    public float[] TimeframeVector { get; set; }
    public float[] PartsCount { get; set; }
    public float[] PartsNoCount { get; set; }
}

IEnumerable<TransformedData> features = mlContext.Data.CreateEnumerable<
    TransformedData>(transformedData, reuseRowObject: false);

Console.WriteLine("Timeframe  TimeframeVector    PartsCount  " +
    "PartsNoCount");

foreach (var featureRow in features)
    Console.WriteLine(featureRow.Timeframe + "          " +
        string.Join(',', featureRow.TimeframeVector.Select(x => x)) + "  "
        + string.Join(',', featureRow.PartsCount.Select(x => x)) +
        "       " + string.Join(',', featureRow.PartsNoCount.Select(
        x => x)));